# Overlapとmp_term_nameの統合（連続値）

- 一例：abnormal_circulating_glucose_level

In [1]:
# Move up to top directory
import os
os.getcwd()
os.chdir('../')
os.getcwd()

'/mnt/g/impc-genes_with_similar_phenotypes'

In [2]:
from pathlib import Path
from collections import defaultdict
from itertools import combinations
import csv
import pandas as pd
from matplotlib import pyplot as plt

In [6]:
path_overlap = Path("data", "overlap_ratios.csv")
df_overlap = pd.read_csv(path_overlap)

In [20]:
path_mp = Path("data", "mp_term_name", "decreased_circulating_glucose_level.csv")
df_mp = pd.read_csv(path_mp)
df_mp = df_mp[["marker_symbol", "p_value", "effect_size"]].drop_duplicates()


In [21]:
print(df_overlap.head(3))

print(len(df_mp))

df_mp

  marker1 marker2  overlap_ratio  overlapped_mp_number  \
0    Dpf2   Gna13          0.167                     2   
1    Dpf2    Plec          0.200                     2   
2    Dpf2  Gnptab          0.080                     2   

                                       overlapped_mp  \
0  abnormal embryo size\npreweaning lethality, co...   
1   edema\npreweaning lethality, complete penetrance   
2  abnormal heart morphology\nabnormal kidney mor...   

                                          marker1_mp  \
0  abnormal craniofacial morphology\nabnormal emb...   
1  abnormal craniofacial morphology\nabnormal emb...   
2  abnormal craniofacial morphology\nabnormal emb...   

                                          marker2_mp  
0  abnormal embryo size\nabnormal visceral yolk s...  
1   edema\npreweaning lethality, complete penetrance  
2  abnormal blood vessel morphology\nabnormal hea...  
166


,marker_symbol,p_value,effect_size
0,Col18a1,3.939500e-05,-1.948883
1,Iqce,4.306803e-05,-1.323595
2,Sptssb,3.314694e-06,-1.393290
3,Agl,0.000000e+00,NaN
4,Dnaja4,1.124332e-05,NaN
...,...,...,...
161,Ttll10,1.203344e-08,-1.466200
162,Enpp5,7.125252e-08,-1.976246
163,Stx18,2.926899e-07,-1.210905
164,Hyal3,8.716088e-05,NaN


In [22]:
# Fill NaN with median

df_mp["effect_size"] = df_mp["effect_size"].fillna(df_mp["effect_size"].median())

In [23]:
print(df_mp["effect_size"].describe())

df_mp[df_mp["effect_size"] < 0]

count    166.000000
mean      -1.333232
std        0.598032
min       -3.762353
25%       -1.563777
50%       -1.300012
75%       -1.021722
max        0.305265
Name: effect_size, dtype: float64


,marker_symbol,p_value,effect_size
0,Col18a1,3.939500e-05,-1.948883
1,Iqce,4.306803e-05,-1.323595
2,Sptssb,3.314694e-06,-1.393290
3,Agl,0.000000e+00,-1.300012
4,Dnaja4,1.124332e-05,-1.300012
...,...,...,...
161,Ttll10,1.203344e-08,-1.466200
162,Enpp5,7.125252e-08,-1.976246
163,Stx18,2.926899e-07,-1.210905
164,Hyal3,8.716088e-05,-1.300012


In [67]:
# print(df_overlap[df_overlap["intersect_count"] > 5 & df_overlap["overlap_ratio"] > 0.5])

In [24]:
# df_overlap_filterd = df_overlap[df_overlap["overlapped_mp_number"] > 0]
df_overlap_filterd = df_overlap[(df_overlap["overlap_ratio"] > 0.5) & (df_overlap["overlapped_mp_number"] > 2)]

In [25]:
df_filtered = df_overlap_filterd[
    (df_overlap_filterd['marker1'].isin(df_mp['marker_symbol'])) &
    (df_overlap_filterd['marker2'].isin(df_mp['marker_symbol']))
]

In [26]:
df_merged = pd.merge(df_filtered, df_mp, how='inner', left_on='marker1', right_on='marker_symbol')

print(len(df_merged))

df_merged.head(3)

29


,marker1,marker2,overlap_ratio,overlapped_mp_number,overlapped_mp,marker1_mp,marker2_mp,marker_symbol,p_value,effect_size
0,Usp32,Pigc,0.667,4,decreased circulating glucose level\ndecreased...,decreased circulating glucose level\ndecreased...,decreased circulating glucose level\ndecreased...,Usp32,2.566989e-10,-1.753421
1,Mtx2,Bud23,0.600,3,decreased circulating glucose level\nembryonic...,decreased circulating glucose level\ndecreased...,decreased circulating glucose level\nembryonic...,Mtx2,4.013782e-05,-1.112288
2,Bud23,Mrm3,0.600,3,decreased circulating glucose level\nembryonic...,decreased circulating glucose level\nembryonic...,abnormal bone structure\ndecreased circulating...,Bud23,9.319209e-06,-0.642843


### 出力

In [27]:
df_network = df_merged[["marker1", "marker2", "effect_size", "overlap_ratio", "marker1_mp", "marker2_mp", "overlapped_mp"]]

df_network.columns = ["from", "to", "node_size", "edge_size", "from_mp", "to_mp", "edge_mp"]
print(len(df_network))
print(df_network.head(3))


29
    from     to  node_size  edge_size  \
0  Usp32   Pigc  -1.753421      0.667   
1   Mtx2  Bud23  -1.112288      0.600   
2  Bud23   Mrm3  -0.642843      0.600   

                                             from_mp  \
0  decreased circulating glucose level\ndecreased...   
1  decreased circulating glucose level\ndecreased...   
2  decreased circulating glucose level\nembryonic...   

                                               to_mp  \
0  decreased circulating glucose level\ndecreased...   
1  decreased circulating glucose level\nembryonic...   
2  abnormal bone structure\ndecreased circulating...   

                                             edge_mp  
0  decreased circulating glucose level\ndecreased...  
1  decreased circulating glucose level\nembryonic...  
2  decreased circulating glucose level\nembryonic...  


In [28]:
df_network.to_csv(Path("data", "network.csv"), index=False)

In [41]:
# %%bash

# python scripts/vis_network.py